In [ ]:
%%capture
! pip install sentence_transformers rank_bm25 nltk

# Objectives

- elaborate a simple semantic search algorithm as it is one of the building block of RAGs.

# Data importing

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample

from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_qq = pd.read_csv("/content/drive/MyDrive/NLP/qora_qestions.csv")
df_qq.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


## Initial data analysis

In [ ]:
# Duplicated questions

df_qq.query('is_duplicate == 1').head()

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [ ]:
# Some data analysis

print("The number of paired questions :", len(df_qq))
print("The number of pairs of duplicated questions :", len(df_qq.query('is_duplicate == 1')))
print("The percentage of pairs of duplicated questions out of the dataset :", round(len(df_qq.query('is_duplicate == 1'))/len(df_qq), 2))

The number of paired questions : 404290
The number of pairs of duplicated questions : 149263
The percentage of pairs of duplicated questions out of the dataset : 0.37


# Retriever Implementation and Evaluation

## Evaluation method

The dataset is composed of sets of pairs of Qora questions with a label indicating if they are to be considered duplicates or not.
An efficient retriever shall retrieve the relevant question out of all the questions.
The retreivers should be evaluated using **at least the Hit Rate and possibly the MAPS.**

## Data structure

In order to evaluate the retrieving methods, you should create the following data sets:
1. A base of unique question with their corresponding ``qid``
2. A key-value dataframe matching ``qid``pairs to a ``is_duplicate`` label

Considering, the number of unique questions (ca 55000) you may subsample the dataset to a manageable size.

## Data preparation code

### Data structures creation

In [ ]:
# Divide question1 and question2
df_1 = df_qq[['qid1', 'question1']].rename(columns={'qid1': 'qid', 'question1': 'question'})
df_2 = df_qq[['qid2', 'question2']].rename(columns={'qid2': 'qid', 'question2': 'question'})

# A base of unique question with their corresponding qid
df_q = pd.concat((df_1, df_2), axis=0).drop_duplicates()

# A key-value dataframe matching qidpairs to a is_duplicate label
df_labels = df_qq[['qid1', 'qid2', 'is_duplicate']]


### Data subsampling

1. Select a set of questions
2. Select the queries as questions from the previous set having at least 1 positive duplicate
3. **Be sure that the ``qid`` of the selected questions are not part of the set of questions that will be indexed in the k-NN for the retriever**

In [ ]:
potential_queries_indices = resample(df_labels.loc[df_labels['is_duplicate'] == 1, 'qid1'].unique(),replace=False, n_samples=1000)
correct_outputs = df_labels.loc[(df_labels['is_duplicate'] == 1)&(df_labels['qid1'].isin(potential_queries_indices)), 'qid2'].unique()

true_positive_basis = set(correct_outputs) - set(potential_queries_indices)
false_examples = resample(df_q.loc[~df_q['qid'].isin(true_positive_basis), 'qid'].values, n_samples=10000, replace=False)

all_ids = np.concatenate((list(true_positive_basis), false_examples))

questions_with_duplicates = df_q.loc[df_q['qid'].isin(potential_queries_indices)]

df_chunks = df_q.loc[df_q['qid'].isin(all_ids)]

### Evaluation functions

Define the relevant functions :
1. match index from questions to labels, by default, if the index pair is not in the labelling table, consider that the the pair is not duplicates.
2. Compute the metrics
3. Evaluation loop over the relevant questions

In [ ]:
def hits(list_labels):
    """
    Calcule le Hit Rate.
    Retourne 1 si au moins une prédiction correcte est trouvée dans list_labels, sinon 0.
    """
    return 1 if any(label == 1 for label in list_labels) else 0

In [ ]:
def ap(list_labels):
    """
    Calcule l'Average Precision.
    """
    precision_sum = 0
    correct_hits = 0
    for i, label in enumerate(list_labels):
        if label == 1:
            correct_hits += 1
            precision_sum += correct_hits / (i + 1)
    return precision_sum / correct_hits if correct_hits > 0 else 0

In [ ]:
def compute_metrics(list_list_labels):
    """
    Calcule les métriques d'évaluation : Hit Rate et Mean Average Precision (MAP).
    """
    hits_list = [hits(l) for l in list_list_labels]
    ap_list = [ap(l) for l in list_list_labels]
    return {
        'hit_rate': sum(hits_list) / len(hits_list),
        'map': sum(ap_list) / len(ap_list)
    }

In [ ]:
test_output = [1,0,0,1,1,0]

print(hits(test_output))#expects 1
print(ap(test_output)) #expects 0.7

1
0.7000000000000001


## Retriever implementation

### First Pipeline

In this first Pipeline we are going to use a simple sklearn pipeline for seamntic search.
#### Steps :
- TfIdf for vectorization
- [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors) class for the search engine

In [ ]:
# tfidf
tfidf_model = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_model.fit_transform(df_chunks['question'])

# vdb
vdb_tfidf = NearestNeighbors(n_neighbors=5, metric='cosine')
vdb_tfidf.fit(tfidf_matrix)

NearestNeighbors(metric='cosine')

In [ ]:
def predict_duplicates_tfidf(query):
  query_vec = tfidf_model.transform([query])
  distances, indices = vdb_tfidf.kneighbors(query_vec)
  return df_chunks.iloc[indices[0]]

qq = questions_with_duplicates.iloc[100]
print('query : \n id: {} question: {}'.format(qq.qid, qq.question))
duplicate_q = df_chunks[df_chunks['qid']==df_labels.loc[(df_labels['is_duplicate']==1) & (df_labels['qid1']==qq.qid), 'qid2'].iloc[0]].iloc[0]
print('duplicate : \n id: {} question: {}'.format(duplicate_q.qid, duplicate_q.question))
predict_duplicates_tfidf(qq['question'])

query : 
 id: 42108 question: Why do people use Quora when they could easily find the answer in a quick Google search?
duplicate : 
 id: 28236 question: Why do people use Quora when they know that they could find a more accurate answer on Google?


,qid,question
56511,26054,Why are there so many people using Quora to an...
68335,82345,Why does people here in Quora still ask some q...
82530,28236,Why do people use Quora when they know that th...
9592,10958,Why do we need to use Quora when we have Googl...
65281,22873,Why should we use Quora when we can Google eve...


#### Evaluation loop

Sample of code, you can change it if you want.

In [ ]:
def get_label(qid1, qid2, df_labels):
    """
    Get the label indicating if two questions are duplicates or not.

    Parameters:
    - qid1, qid2: Question IDs to compare.
    - df_labels: DataFrame with columns ['qid1', 'qid2', 'is_duplicate'].

    Returns:
    - 1 if duplicate, 0 if not duplicate.
    """
    label = df_labels.loc[((df_labels['qid1'] == qid1) & (df_labels['qid2'] == qid2)) |
                          ((df_labels['qid1'] == qid2) & (df_labels['qid2'] == qid1)), 'is_duplicate']
    return label.iloc[0] if not label.empty else 0

In [ ]:
all_labels = []

for id, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
  query = question['question']
  results = predict_duplicates_tfidf(query)
  labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
  all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 0.96, 'map': 0.8826194444444442}

### Second Pipeline : Sentence transformer pipeline

#### Steps:
 - Use the sentence_transformers package to load a model and embed the reviews
 - Since it is much more demanding than TfIdf, it could be necessary to subsample the dataset randomly
 - Build a pipeline with the same NearestNeighbors engine.

In [ ]:
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Use Batch Encoding
def batch_encode(questions, model, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(questions), batch_size)):
        batch = questions[i:i + batch_size]
        batch_embeddings = model.encode(batch)
        embeddings.extend(batch_embeddings)
    return embeddings


# Generate embeddings for all questions
question_embeddings = batch_encode(df_chunks['question'].tolist(), sbert_model)

  0%|          | 0/368 [00:00<?, ?it/s]

In [ ]:
# Initialize NearestNeighbors model and fit it with the embeddings
vdb_sbert = NearestNeighbors(n_neighbors=5, metric='cosine')
vdb_sbert.fit(question_embeddings)

NearestNeighbors(metric='cosine')

In [ ]:
def predict_duplicates_sbert(query):
    query_vec = sbert_model.encode([query])  # Encode the query
    distances, indices = vdb_sbert.kneighbors(query_vec)  # Find nearest neighbors
    return df_chunks.iloc[indices[0]]  # Return corresponding rows from the DataFrame

In [ ]:
qq = questions_with_duplicates.iloc[100]
print('query : \n id: {} question: {}'.format(qq.qid, qq.question))
duplicate_q = df_chunks[df_chunks['qid']==df_labels.loc[(df_labels['is_duplicate']==1) & (df_labels['qid1']==qq.qid), 'qid2'].iloc[0]].iloc[0]
print('duplicate : \n id: {} question: {}'.format(duplicate_q.qid, duplicate_q.question))
predict_duplicates_sbert(qq['question'])

query : 
 id: 42108 question: Why do people use Quora when they could easily find the answer in a quick Google search?
duplicate : 
 id: 28236 question: Why do people use Quora when they know that they could find a more accurate answer on Google?


,qid,question
56511,26054,Why are there so many people using Quora to an...
82530,28236,Why do people use Quora when they know that th...
13575,26053,Why do so many people ask things on Quora that...
9592,10958,Why do we need to use Quora when we have Googl...
15346,29331,Why don't Quora people just look up the answer...


In [ ]:
all_labels = []
for _, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
    query = question['question']
    results = predict_duplicates_sbert(query)
    labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
    all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 0.996, 'map': 0.9581333333333332}

### Third Pipleine: BM25

After using distance based searchs methodfs, we are going to work with the BM25 algorithm. It was developped in late 70s but it is still very relevant. Some preprocessing of the text will be needed. A classical pipeline will be implemented.
#### Preprocessing

##### Stopwords, Stemming

In [ ]:
# Initialize the stemmer and stopwords

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

### Queries

#### Preprocessing

In [ ]:
# Define Preprocessing Function
def query_preprocessing(query):
    query = query.lower() # Lowercase the text
    query = query.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    tokens = query.split() # Tokenize the text
    processed_query = [stemmer.stem(word) for word in tokens if word not in stop_words]  # Remove stopwords and perform stemming
    return processed_query

# Preprocess all questions in the dataset
df_chunks = df_chunks.copy()
df_chunks['processed_question'] = df_chunks['question'].apply(query_preprocessing)

In [ ]:
df_chunks

,qid,question,processed_question
87,175,What is the difference between sincerity and f...,"[differ, sincer, fair]"
106,213,Have you ever heard of travel hacking?,"[ever, heard, travel, hack]"
165,331,Which is the best earphone with deep bass unde...,"[best, earphon, deep, bass, 1000]"
203,407,Why do people hate Hillary Clinton?,"[peopl, hate, hillari, clinton]"
229,459,How can we make the world a better place to li...,"[make, world, better, place, live, futur, gene..."
...,...,...,...
404038,537666,What is the ranking of UCF for MS in CS?,"[rank, ucf, ms, cs]"
404072,537707,How can I add a non-iPhone video to the iPhone...,"[add, noniphon, video, iphon, camera, roll]"
404119,537764,Is Gaza going to run out of fresh water?,"[gaza, go, run, fresh, water]"
404129,537773,How can I get the outgoing?,"[get, outgo]"


#### BM25 engine

In [ ]:
# Create the BM25 Okapi engine

bm25_model = BM25Okapi(df_chunks['processed_question'].tolist())

In [ ]:
def predict_duplicates_bm25(query):
    # Preprocess the input query
    preprocessed_query = query_preprocessing(query)
    # Get the top 5 matching documents based on BM25 ranking
    best_matches = bm25_model.get_top_n(preprocessed_query, df_chunks['question'].tolist(), n=5)
    # Return the matching rows from the DataFrame
    return df_chunks[df_chunks['question'].isin(best_matches)]

In [ ]:
qq = questions_with_duplicates.iloc[100]
print('query : \n id: {} question: {}'.format(qq.qid, qq.question))
duplicate_q = df_chunks[df_chunks['qid']==df_labels.loc[(df_labels['is_duplicate']==1) & (df_labels['qid1']==qq.qid), 'qid2'].iloc[0]].iloc[0]
print('duplicate : \n id: {} question: {}'.format(duplicate_q.qid, duplicate_q.question))
predict_duplicates_bm25(qq['question'])

query : 
 id: 42108 question: Why do people use Quora when they could easily find the answer in a quick Google search?
duplicate : 
 id: 28236 question: Why do people use Quora when they know that they could find a more accurate answer on Google?


,qid,question,processed_question
9592,10958,Why do we need to use Quora when we have Googl...,"[need, use, quora, googl, search, answer]"
39789,12639,Why do people ask stupid questions on Quora th...,"[peopl, ask, stupid, question, quora, could, e..."
56511,26054,Why are there so many people using Quora to an...,"[mani, peopl, use, quora, answer, question, ea..."
67612,38,Why do people ask Quora questions which can be...,"[peopl, ask, quora, question, answer, easili, ..."
82530,28236,Why do people use Quora when they know that th...,"[peopl, use, quora, know, could, find, accur, ..."


In [ ]:
all_labels = []

for id, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
  query = question['question']
  results = predict_duplicates_bm25(query)
  labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
  all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 0.982, 'map': 0.5003944444444445}

# Conclusion

**1. Comparison of the Different Methods:**

In the results, the three pipelines are evaluated based on two key performance metrics: hit rate and MAP (Mean Average Precision).

- **First Pipeline: Simple sklearn pipeline for semantic search**

  **Hit rate: 0.96 | MAP: 0.8826**

  This pipeline performs very well, with a high hit rate and a strong MAP score. The method used here might be based on standard TF-IDF vectors with a nearest-neighbor search algorithm(k-NN).

- **Second Pipeline: Sentence transformer pipeline**

  **Hit rate: 0.996 | MAP: 0.9581**

  This pipeline performs the best across both metrics. The use of Sentence Transformers, which are based on pre-trained deep learning models, provides a semantic understanding of the text. This allows it to capture deeper contextual relationships between sentences or documents, resulting in a much higher hit rate and MAP than the first pipeline.

- **Third Pipeline: BM25**

  **Hit rate: 0.982 | MAP: 0.5004**

  BM25 performs well in terms of hit rate, but its MAP score is significantly lower than the first and second pipelines. BM25 is an information retrieval model based on probabilistic matching, typically using term frequency and document frequency to rank documents. It seems to be less effective in capturing semantic relationships, as reflected in the lower MAP score, which considers the ranking order of relevant documents.

**2. Explanation of the Difference between BM25 and TF-IDF (in k-NN context):**

- **BM25 vs. TF-IDF:**

  BM25 is a probabilistic retrieval model that uses document frequency (IDF) and term frequency (TF), but it introduces a non-linear scaling factor, which can help in reducing the dominance of very frequent terms in longer documents. BM25 also incorporates parameters like k1 (controls term frequency) and b (controls the length normalization), which are designed to improve retrieval effectiveness in terms of ranking documents.

  TF-IDF (Term Frequency-Inverse Document Frequency), on the other hand, is more deterministic. It weighs terms based on their frequency within a document relative to their frequency across the entire corpus. In traditional k-NN with TF-IDF vectors, document vectors are compared based on cosine similarity, without considering deeper semantic relationships. This can be less flexible and less effective when it comes to understanding context or nuances in language.

**Why BM25 performs worse in terms of MAP:**

- Semantic understanding: BM25 focuses on term frequency and inverse document frequency, which does not capture the meaning or semantic similarity of words as well as other methods like Sentence Transformers. This can result in poorer ranking (lower MAP), even though it still retrieves relevant documents in the top results (high hit rate).

- Ranking sensitivity: Since MAP evaluates how well the retrieved documents are ranked in order of relevance, BM25's ranking might not be as sensitive to nuanced semantic relationships between documents as models like sentence transformers or even TF-IDF with k-NN.


**Why TF-IDF (in k-NN) outperforms BM25 in MAP:**
- In k-NN, TF-IDF vectors are used to compute similarity scores (typically cosine similarity) between query and document vectors. These vectors are directly comparable in a high-dimensional space, and the more relevant the terms are to the query, the higher the similarity score. This often works better for ranking, which leads to a higher MAP than BM25, as TF-IDF-based methods can rank semantically relevant documents better, even though they lack a true semantic understanding.


In summary, sentence transformers outperform both BM25 and TF-IDF methods because they better capture semantic meaning, while BM25 performs decently in retrieval but is not as sensitive to ranking relevance based on semantic content, leading to a lower MAP score. TF-IDF in a k-NN setup can still capture useful similarity for ranking, explaining its superior performance compared to BM25 in terms of MAP.
